In [1]:
import sys
import os

sys.path.append(sys.path[0]+"/tracker")
sys.path.append(sys.path[0]+"/tracker/model")

from track_anything import TrackingAnything
import random 
import string
from dataset.longdataset import LongVideoDataset
from tools.runner import run_model_on_longdata_set


In [2]:
# Change path to .datasets own folder 
if os.name == 'posix':
    longdataset_root = '../data.nosync/LongDataset/'

else:
    longdataset_root = 'D:/HADA/data/LongDataset/'


In [3]:
SAM_checkpoint = "./checkpoints/sam_vit_h_4b8939.pth"
if 'HQ' in sys.prefix.split('.')[-1]:SAM_checkpoint = "./checkpoints/sam_hq_vit_h.pth"
xmem_checkpoint = "./checkpoints/XMem-s012.pth"

In [ ]:
runtimeargs_lst = [{
    'DatasetArgs' :{
        'Dataset' : 'LongDataset',
        'Set' : 'val',
    },
    'TrackingAnythingArgs' : {
            'use_refinement' : False,
            'refinement_mode' : 'both_neg',
            'addArgs1':'CP',
            'optimized':False,
    }
},{
    'DatasetArgs' :{
        'Dataset' : 'LongDataset',
        'Set' : 'val',
    },
    'TrackingAnythingArgs' : {
            'use_refinement' : True,
            'refinement_mode' : 'both_neg',
            'addArgs1':'CP',
            'optimized':False,
    }
},{
    'DatasetArgs' :{
        'Dataset' : 'LongDataset',
        'Set' : 'val',
    },
    'TrackingAnythingArgs' : {
            'use_refinement' : True,
            'refinement_mode' : 'both_neg',
            'addArgs1':'CP',
            'optimized':True,
    }
}]

VideoLoader = LongVideoDataset(root_folder=longdataset_root, filelist= runtimeargs_lst[0]['DatasetArgs']['Set']+'.txt')

for runtimeargs in runtimeargs_lst:
    runname = '{}{}{}{}_{}_{}_{}'.format(
        'Refined' if runtimeargs['TrackingAnythingArgs']['use_refinement']  else 'XMEM',
        '_HQ' if 'HQ' in sys.prefix.split('.')[-1] and runtimeargs['TrackingAnythingArgs']['use_refinement'] else '_SAM' if runtimeargs['TrackingAnythingArgs']['use_refinement'] else  '',
        '_'+ runtimeargs['TrackingAnythingArgs']['refinement_mode'] if runtimeargs['TrackingAnythingArgs']['use_refinement'] else '',
        '_' + runtimeargs['TrackingAnythingArgs']['addArgs1'] if runtimeargs['TrackingAnythingArgs']['use_refinement'] and runtimeargs['TrackingAnythingArgs']['addArgs1'] != '' else '',
        'Optimized' if runtimeargs['TrackingAnythingArgs']['use_refinement'] and runtimeargs['TrackingAnythingArgs']['optimized'] else 'NONOptimized' if runtimeargs['TrackingAnythingArgs']['use_refinement'] else '',
        'Davis_{}_{}_{}'.format(runtimeargs['DatasetArgs']['Year'],runtimeargs['DatasetArgs']['Set'],runtimeargs['DatasetArgs']['Resolution']) if runtimeargs['DatasetArgs']['Dataset'] == 'Davis' else runtimeargs['DatasetArgs']['Dataset'],
        ''.join(random.choice(string.ascii_letters) for _ in range(5))
    )
    print(f'Running Test: {runname}')
    model = TrackingAnything(SAM_checkpoint, xmem_checkpoint, None,runtimeargs['TrackingAnythingArgs'], save_inner_masks_folder = runname)
    masks, logits, painted_images = run_model_on_longdata_set(name = runname,model = model,videoLoader = VideoLoader,compute_metrics = True, save_masks=False, compute_video=False,verbose = False)